In [1]:
import findspark
findspark.init('C:\\spark\spark-3.2.1-bin-hadoop3.2')  
import pyspark
from pyspark.sql import SparkSession 
from pyspark import SparkContext, SQLContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from  datetime import datetime
import os
import shutil
import time


checkpointDir = "./CheckPoint"
path = "./result"

# очищаем директорию чекпоинтов
try:
    shutil.rmtree(checkpointDir)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))


# очищаем директорию чекпоинтов пути сохранения
try:
    shutil.rmtree(path)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror)) 

csvSchema = "symbol String, price Float"
    
spark = (SparkSession
 .builder
 .appName('crypto')
 .enableHiveSupport()
 .getOrCreate())

# Открытие потока на чтение
crypto_currency = (spark
  .readStream
  .format("csv")
  .option("delimiter", ";")
  .schema(csvSchema)
  .load("./source"))

# Добавляю столбец с датой и временем прихода данных
crypto_currency = crypto_currency \
    .withColumn('sampling_datetime', lit(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))


# Запись измененного потока в директорию в формате json
streamingQuery = (crypto_currency             
    .repartition(1)    
    .writeStream
    .outputMode("append")
    .format("json")
    .option("checkpointLocation", checkpointDir) 
    .trigger(processingTime="5 second")
    .start(path))

streamingQuery.awaitTermination()    